In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.autograd import Function
def get_parameter_number(net):
    total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}

dtype = torch.float

class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, *size ,dtype = dtype)
        self.label = torch.rand(1, dtype = dtype)

    def __getitem__(self, index):
        return self.data[index], self.label

    def __len__(self):
        return self.len

        
def Random_DataLoader():
    input_size = [3]

    train_img_loader = DataLoader(dataset=RandomDataset(input_size, length = 100),
                         batch_size=2, shuffle=True)
    val_img_loader =  DataLoader(dataset=RandomDataset(input_size, length = 10),
                         batch_size=2, shuffle=False)

    return train_img_loader, val_img_loader


trainloader, valloader =  Random_DataLoader()

In [347]:
class LinearFunction(torch.autograd.Function):
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        print(input.size())
        print(weight.size())
        print(output.size())
        
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
            
        return output

    @staticmethod
    def backward(ctx, grad_output):
#         print(grad_output)
        input, weight, bias = ctx.saved_tensors
        
        grad_input = grad_weight = grad_bias = None

        
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)

            
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)

                 
        if bias is not None and ctx.needs_input_grad[2]:
            
            grad_bias = grad_output.sum(0).squeeze(0)
            grad_bias = grad_bias.view_as(bias)
#         print("*" * 40)
#         print(bias.size())
#         print(grad_weight.size())
#         print(grad_bias.size())
#         print("*" * 40)
        return grad_input, grad_weight, grad_bias



In [348]:
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            self.register_parameter('bias', None)
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None)

In [350]:
demo_net = Linear(3,1)

criterion = nn.MSELoss()
optimizer = optim.SGD(demo_net.parameters(), lr=0.001, momentum=0.9)


for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        
        inputs, labels = data
        optimizer.zero_grad()

        outputs = demo_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        break
print('Finished Training')

torch.Size([2, 3])
torch.Size([1, 3])
torch.Size([2, 1])
Finished Training


In [ ]:
### test 2d transform matrix

In [356]:
input_matrix = torch.rand(2, 6)

kernel_size = 3
input_channels = 2
output_channels = 2
ks = kernel_size // 2
per_col = len(ls) - ks -1
transform_maxtrix = torch.zeros([per_col * 2 , kernel_size])
fil  = torch.rand([3,output_channels])


In [358]:
for index in range(len(input_matrix)):
    ls = input_matrix[index,:]
    for i in range(ks, len(ls) - ks):
        transform_maxtrix[i-1 + index * per_col,:] = ls[i-ks: i+ks+1]


output_matrix = transform_maxtrix.mm(fil)
output = output.T
output.size()

torch.Size([2, 4])

In [359]:
### test reshape order
a = np.array([[[111,112],[121,122]],[[211,212],[221,222]]])
a.reshape(2,-1, order = 'F')

array([[111, 121, 112, 122],
       [211, 221, 212, 222]])

In [368]:
### test reshape order
a = torch.rand([2,3,4,5])
b = a.reshape(-1,a.size(3))
# print(a)
# print(b)

In [364]:
### test 4d transform matrix
def matrix3d_matrix1d(input_matrix, kernel_size ):
    
    input_row = input_matrix.size(0)
    input_col = input_matrix.size(1)
    
    ks = kernel_size // 2
    trans_subrow = len(input_matrix[0]) - ks -1
    trans_row = trans_subrow * input_row
    trans_col = kernel_size
    transform_maxtrix = torch.zeros([trans_row , trans_col])
    
    for index in range(input_row):
        
        ls = input_matrix[index,:]
        
        for i in range(ks, len(ls) - ks):
            transform_maxtrix[i-1 + index * trans_subrow,:] = ls[i-ks: i+ks+1]
    return transform_maxtrix

def myconv1d(input_m, kernel,kernel_size):
    # batch_size, in_channels, in_row, in_col
    
    temp_m = input_m.reshape(-1,input_m.size(1))
    res = matrix3d_matrix1d(temp_m, kernel_size )
    
    res = res.mm(kernel)
    res = res.reshape(input_m.size(0),-1,input_m.size(2),kernel.size(1))
    
    return res 


In [365]:
### test 4d transform matrix

kernel_size = 3
output_channels = 1
input_matrix = torch.rand(2, 6, 6, 3)
# batch_size, in_channels, in_row, in_col
# batch_size, in_x, in_y, in_z

kernel  = torch.rand([kernel_size,output_channels])

d= myconv1d(input_matrix, kernel,kernel_size)
d.size()


torch.Size([2, 12, 6, 1])

In [23]:
def conv(img, conv_filter):
   
    if len(img.shape)!=3 or len(conv_filter.shape)!=4:
        print("卷积运算所输入的维度不符合要求")
        sys.exit()
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("卷积输入图片与卷积核的通道数不一致")
        sys.exit()
        
    img_h, img_w, img_ch = img.shape
    filter_num, filter_h, filter_w, img_ch = conv_filter.shape
    feature_h = img_h - filter_h + 1
    feature_w = img_w - filter_w + 1

    # 初始化输出的特征图片，由于没有使用零填充，图片尺寸会减小
    img_out = np.zeros((feature_h, feature_w, filter_num))
    img_matrix = np.zeros((feature_h*feature_w, filter_h*filter_w*img_ch))
    filter_matrix = np.zeros((filter_h*filter_w*img_ch, filter_num))
    
    # 将输入图片张量转换成矩阵形式
    for i in range(feature_h*feature_w):
        for j in range(img_ch):
            img_matrix[i, j*filter_h*filter_w:(j+1)*filter_h*filter_w] = \
            img[np.uint16(i/feature_w):np.uint16(i/feature_w+filter_h),np.uint16(i%feature_w):np.uint16(i%feature_w+filter_w),j].reshape(filter_h*filter_w)
    
    # 将卷积核张量转换成矩阵形式
    for i in range(filter_num):
        filter_matrix[:,i] = conv_filter[i,:].reshape(filter_w*filter_h*img_ch)
    
    feature_matrix = np.dot(img_matrix, filter_matrix)
    
    for i in range(filter_num):
        img_out[:,:,i] = feature_matrix[:,i].reshape(feature_h, feature_w)
    
    return img_out


In [243]:
input_matrix

tensor([[0.3045, 0.3638, 0.1190, 0.0709, 0.3345, 0.7904],
        [0.6410, 0.8664, 0.9376, 0.1281, 0.6148, 0.0498]])

In [211]:
transform_maxtrix

tensor([[0.8109, 0.4977, 0.8461],
        [0.4977, 0.8461, 0.6593],
        [0.8461, 0.6593, 0.6308],
        [0.6593, 0.6308, 0.4467],
        [0.2395, 0.0494, 0.8369],
        [0.0494, 0.8369, 0.6761],
        [0.8369, 0.6761, 0.1070],
        [0.6761, 0.1070, 0.8767]])

In [ ]:
def conv2d_3conv1d

In [339]:
class flatten_conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, in_row, in_col,
                kernel_size, stride=1, padding=0, 
                dilation=1, groups=1, bias=True, padding_mode='zeros'):

        super(flatten_conv2d, self).__init__()
        
        self.in_row = in_row
        self.in_col = in_col
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        self.L1_weight = nn.Parameter(torch.Tensor(kernel_size, out_channels))
        self.L1_bias = nn.Parameter(torch.Tensor(out_channels))

        
        self.L1_weight.data.uniform_(-0.1, 0.1)
        self.L1_bias.data.uniform_(-0.1, 0.1)
        
    def forward(self, input):
#         x = self.matrix3d_matrix1d(input, self.kernel_size )
        # See the autograd section for explanation of what happens here.
        res = test_fun.apply(input, self.L1_weight, self.L1_bias)
#         res = input
        return res
    
    def matrix3d_matrix1d(self, input_matrix, kernel_size ):

        input_row = input_matrix.size(0)
        input_col = input_matrix.size(1)
        
        ks = kernel_size // 2
        trans_subrow = len(input_matrix[0]) - ks -1
        trans_row = trans_subrow * input_row
        trans_col = kernel_size
        transform_maxtrix = torch.zeros([trans_row , trans_col])

        for index in range(input_row):

            ls = input_matrix[index,:]

            for i in range(ks, len(ls) - ks):
                transform_maxtrix[i-1 + index * trans_subrow,:] = ls[i-ks: i+ks+1]
        return transform_maxtrix


#     def extra_repr(self):
#         # (Optional)Set the extra information about this module. You can test
#         # it by printing an object of this class.
#         return 'in_features={}, out_features={}, bias={}'.format(
#             self.in_features, self.out_features, self.bias is not None)

In [340]:

kernel_size = 3
output_channels = 1
input_matrix = torch.rand(1, 2, 6, 3)

f = flatten_conv2d(in_channels=2, out_channels=1, in_row=3, in_col=3,
                kernel_size=3)

r = f(input_matrix)
r.size()

torch.Size([1, 2, 6, 3])


torch.Size([1, 2, 6, 3])

In [338]:
class test_fun(torch.autograd.Function):
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias):
        ctx.save_for_backward(input, weight, bias)
        print(input.size())
        return input

    @staticmethod
    def backward(ctx, grad_output):
#         print(grad_output)
        input, weight, bias = ctx.saved_tensors
        
        grad_input = grad_weight = grad_bias = None
        
        return grad_input, grad_weight, grad_bias



In [318]:
conv1 = nn.Conv1d(2,3,kernel_size)
input_matrix = torch.rand(2, 2, 6)
conv1(input_matrix).size()

torch.Size([2, 3, 4])

torch.Size([2, 4, 3, 1])

In [306]:
c = c.reshape(2,-1,3)
c

tensor([[[0.7209, 0.7582, 0.7923],
         [1.2069, 1.3946, 0.9410],
         [1.5652, 1.2715, 0.5333],
         [0.8409, 0.9237, 1.5476]],

        [[1.4687, 1.5621, 1.1674],
         [0.8757, 0.8300, 1.1598],
         [0.8386, 1.1064, 0.9700],
         [0.9292, 1.1963, 1.6553]]])

In [290]:
d = matrix3d_matrix1d(c,kernel_size)
myconv1d(d, kernel)

tensor([[0.4304],
        [0.2370]])

In [ ]:
myconv1d(trans_m, kernel)